In [122]:
import pandas as pd

In [123]:
test = pd.read_csv('test.csv', encoding = 'cp949')
train = pd.read_csv('train.csv', encoding = 'cp949')
sub = pd.read_csv('sample_submission.csv', encoding = 'cp949')

- target
    - 전력사용량(kWh)
- feature
    - numeric
        - 기온
        - 풍속
        - 습도
        - 강수량
        - 일조
        - 비전기냉방시설운영 : 0,1 factor
        - 태양광보유 : 0,1 factor
        - num : 건물번호
    - datetime
        - date_time

- 비가 내리고 안내리고에 따라서 태양광을 보유한 곳에서 전기 사용량 차이가 있는지 확인

In [124]:
feature_cols = ['기온(°C)','풍속(m/s)','습도(%)','일조(hr, 3시간)','강수량(mm, 6시간)']

for i in range(4):
    for x in range(0,len(test),3):
        for j in range(1,3):
            test[feature_cols[i]][x+j] = test[feature_cols[i]][x]

for x in range(0,len(test),6):
    for k in range(1,6):
        test['강수량(mm, 6시간)'][x+k] = test['강수량(mm, 6시간)'][x]


<ipython-input-124-51685eb8f665>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[feature_cols[i]][x+j] = test[feature_cols[i]][x]
<ipython-input-124-51685eb8f665>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['강수량(mm, 6시간)'][x+k] = test['강수량(mm, 6시간)'][x]


In [125]:
train.loc[train['강수량(mm)'] !=0, '우천'] = 1
train.loc[train['강수량(mm)'] ==0, '우천'] = 0

test.loc[test['강수량(mm, 6시간)'] !=0, '우천'] = 1
test.loc[test['강수량(mm, 6시간)'] ==0, '우천'] = 0



In [126]:

#date_time
train['month'] = pd.to_datetime(train['date_time']).dt.month
train['day'] = pd.to_datetime(train['date_time']).dt.day
train['hour'] = pd.to_datetime(train['date_time']).dt.hour


test['month'] = pd.to_datetime(test['date_time']).dt.month
test['day'] = pd.to_datetime(test['date_time']).dt.day
test['hour'] = pd.to_datetime(test['date_time']).dt.hour

In [127]:
# import statsmodels.api as sm
# feature_cols = train.columns.difference(['date_time','전력사용량(kWh)'])
# x = train[feature_cols]
# y = train[['전력사용량(kWh)']]
# X = sm.add_constant(x, has_constant= 'add')
# model = sm.OLS(y, X)
# multi_model = model.fit()
# multi_model.summary()

In [128]:
train.loc[(train['hour'] > 5 ) & (train['hour'] < 19) , '해 떠있는 시간'] = 1
train.loc[(train['hour'] <= 5 ) | (train['hour'] >= 19) , '해 떠있는 시간'] = 0


test.loc[(test['hour'] > 5 ) & (test['hour'] < 19) , '해 떠있는 시간'] = 1
test.loc[(test['hour'] <= 5 ) | (test['hour'] >= 19) , '해 떠있는 시간'] = 0

In [129]:
test['비전기냉방설비운영'].fillna(0, inplace =True)
test['태양광보유'].fillna(0, inplace = True)

In [130]:
test.rename(columns = {'강수량(mm, 6시간)' : '강수량(mm)', '일조(hr, 3시간)' : '일조(hr)'},inplace = True)

In [131]:
# testnum_cols = ['기온(°C)','풍속(m/s)','습도(%)','강수량(mm)','일조(hr)']

# from sklearn.preprocessing import StandardScaler
# sds = StandardScaler()
# train[num_cols] = sds.fit_transform(train[num_cols])
# test[num_cols] = sds.fit_transform(test[num_cols])

In [132]:
feature_cols = train.columns.difference(['date_time','전력사용량(kWh)'])
x_train = train[feature_cols]
y_train = train[['date_time','전력사용량(kWh)']]

x_test = test[feature_cols]

In [133]:
x_train.shape, y_train.shape, x_test.shape, sub.shape

((122400, 13), (122400, 2), (10080, 13), (10080, 2))

In [134]:
from lightgbm import LGBMRegressor
lgbm = LGBMRegressor().fit(x_train, y_train['전력사용량(kWh)'])
y_pred = lgbm.predict(x_test)
sub['answer'] = y_pred

In [135]:
sub.to_csv('result_only_use_lgbm.csv', index = False)